# **Grouped `credit_card_balance`**

# Data Loading and Preprocessing

**Time:** 1.4 s for 3,840,312 entries.

## New

In [9]:
from home_credit.tables import CreditCardBalance
from home_credit.utils import display_frame_basic_infos
from pepper.univar import print_value_counts_dict

data = CreditCardBalance.clean()
display_frame_basic_infos(data)
print_value_counts_dict(data, "NAME_CONTRACT_STATUS")
display(data)

n_samples: 3 840 312
n_columns: 21, [('SK', 2), ('NAME', 1), ('CNT', 5), ('AMT', 12), ('TARGET', 1)]
NAME_CONTRACT_STATUS (7): {'Active': 3698436, 'Completed': 128918, 'Signed': 11058, 'Demand': 1365, 'Sent proposal': 513, 'Refused': 17, 'Approved': 5}


CLEAN_CREDIT_CARD_BALANCE             TARGET  AMT_BALANCE  ...  SK_DPD  SK_DPD_DEF
SK_ID_CURR SK_ID_PREV MONTHS_BALANCE                       ...                    
100006     1489396    1                    0        0.000  ...       0           0
                      2                    0        0.000  ...       0           0
                      3                    0        0.000  ...       0           0
                      4                    0        0.000  ...       0           0
                      5                    0        0.000  ...       0           0
...                                      ...          ...  ...     ...         ...
456250     1794451    8                   -1   177219.000  ...       0           0
                      9                   -1   180536.760  ...       0           0
                      10                  -1   186577.605  ...       0           0
                      11                  -1   200208.915  ...       0           0
                      12                  -1   181993.500  ...       0           0

[3840312 rows x 21 columns]

## Old

In [4]:
from home_credit.load import get_table
from pepper.utils import display_key_val

# Load the 'credit_card_balance' table
data = get_table("credit_card_balance").copy()

# Adjust the 'MONTHS_BALANCE' column to ensure consistency
data.MONTHS_BALANCE = -data.MONTHS_BALANCE

# Insert the aggregation counter
data.insert(0, "n_PREV", 1)

# Display the number of samples in the dataset
display_key_val("number of samples", data.shape[0])

# Display the dataset
display(data)

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\credit_card_balance.pqt
number of samples: 3 840 312


RAW_CREDIT_CARD_BALANCE,n_PREV,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1,2562384,378907,6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.00,1800.000,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,1,2582071,363914,1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.00,2250.000,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1,1740877,371185,7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.00,2250.000,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1,1389973,337855,4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.00,11925.000,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1,1891521,126868,1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.00,27000.000,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1,1036507,328243,9,0.000,45000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0
3840308,1,1714892,347207,9,0.000,45000,0.0,0.0,0.0,0.0,0.000,1879.11,0.000,0.000,0.000,0.000,0.0,0,0.0,0.0,23.0,Active,0,0
3840309,1,1302323,215757,9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,375750.00,356994.675,269356.140,273093.975,273093.975,2.0,2,0.0,0.0,18.0,Active,0,0
3840310,1,1624872,430337,10,0.000,450000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0


# Key Uniqueness

We verify that there cannot be multiple `SK_ID_CURR` for one `SK_ID_PREV`.

The issue is therefore multi-indexed only in appearance: the `SK_ID_CURR` key is sufficient to separate the groups.

Number of `SK_ID_PREV` for one `SK_ID_CURR` and vice versa :

In [6]:
from home_credit.merge import _get_unique_and_multi_index, curr_prev_uniqueness_report

# Get unique and multi-indexes for the specified table and columns
indexes = _get_unique_and_multi_index(data.reset_index(), "SK_ID_PREV", "SK_ID_CURR")

# Generate a report on the uniqueness of SK_ID_CURR and SK_ID_PREV
curr_prev_uniqueness_report(*indexes)

number of unique (curr, prev)              : 104 307
number of curr with more than 1 prev       : 1 489
number of curr with one prev               : 102 818
number of curr with more than 1 prev (in %): 1.4
number of prev with more than 1 curr       : 0
number of prev with one curr               : 104 307
number of prev with more than 1 curr (in %): 0.0


# Agrégation cf. **`old_kernel_v2`**

Le premier jet était inspiré du **`lightgbm_kernel`**, un kernel de référence disponible sur Kaggle.

Il s'agit d'une agrégation par prêt qui produit 103 558 échantillons de synthèse.

L'information est appauvrie, et propose uniquement ces indicateurs pour chaque variable, sans distinction de nature : minimum, maximum, moyenne, somme et variance.

In [8]:
agg_rule = ["min", "max", "mean", "sum", "var"]
grouped = data.groupby('SK_ID_CURR')
aggregated = grouped.agg(agg_rule)
aggregated["CC_COUNT"] = grouped.size()
display(aggregated)

C:\Users\franc\AppData\Local\Temp\ipykernel_22844\1680920085.py:3: FutureWarning: ['NAME_CONTRACT_STATUS'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  aggregated = grouped.agg(agg_rule)


CLEAN_CREDIT_CARD_BALANCE TARGET               ... SK_DPD_DEF               CC_COUNT
                             min max mean sum  ...       mean sum       var         
SK_ID_CURR                                     ...                                  
100006                         0   0  0.0   0  ...   0.000000   0  0.000000        6
100011                         0   0  0.0   0  ...   0.000000   0  0.000000       74
100013                        -1  -1 -1.0 -96  ...   0.010417   1  0.010417       96
100021                         0   0  0.0   0  ...   0.000000   0  0.000000       17
100023                         0   0  0.0   0  ...   0.000000   0  0.000000        8
...                          ...  ..  ...  ..  ...        ...  ..       ...      ...
456244                         0   0  0.0   0  ...   0.000000   0  0.000000       41
456246                         0   0  0.0   0  ...   0.000000   0  0.000000        8
456247                         0   0  0.0   0  ...   0.021053   2  0.020829       95
456248                         0   0  0.0   0  ...   0.000000   0  0.000000       23
456250                        -1  -1 -1.0 -12  ...   0.000000   0  0.000000       12

[103558 rows x 101 columns]

# Profils denses de variation à l'aide de l'encodage RLE

Nous inspirant du cas **`bureau_balance`**, nous commençons par cette opération qui a l'avantage de rendre apparents les _patterns_ dynamiques de chaque variable, qu'elle soit numérique ou catégorielle.

Par commodité, nous allons ré-encoder avec une seule lettre chacune  modalités de `NAME_CONTRACT_STATUS`.

## Période de suivi

La période de suivi d'un prêt peut être la totalité des 96 mois, soit une sous-période, voire des sous-périodes fragmentées.

Notre première fonction, basée sur la fonction `jumps_rle` permet de codifier en RLE les sauts entre mois consécutifs de suivi.

### Période de suivi par prêt

On produit la table indexée par `SK_ID_PREV` des périodes de suivi des prêts, avec le premier, le dernier et le nombre de mois de suivi, et la représentation RLE des sous-périodes (dans la plupart des cas, une seule).

In [2]:
from home_credit.tables import CreditCardBalance

tracking = CreditCardBalance.rle_loan_tracking_period()
display(tracking)
# ok tracking.info()

CLEAN_POS_CASH_BALANCE MONTHS_BALANCE                              
                                  min max count           jumps_rle
SK_ID_PREV                                                         
1000001                             8  10     3    [[9, 1], [1, 2]]
1000002                            50  54     5   [[51, 1], [1, 4]]
1000003                             1   4     4    [[2, 1], [1, 3]]
1000004                            22  29     8   [[23, 1], [1, 7]]
1000005                            46  56    11  [[47, 1], [1, 10]]
...                               ...  ..   ...                 ...
2843494                            24  26     3   [[25, 1], [1, 2]]
2843495                             9  16     8   [[10, 1], [1, 7]]
2843497                             1  21    21   [[2, 1], [1, 20]]
2843498                            42  48     7   [[43, 1], [1, 6]]
2843499                            30  40    11  [[31, 1], [1, 10]]

[935435 rows x 4 columns]